## Procesamiento de Lenguaje Natural

En este notebook veremos dos técnicas básicas, aunque en muchos casos eficientes, para codificar la información contenida en los textos que queremos analizar. Estas técnicas se conocen como bolsa de palabras (o Bag-of-words) y TF-IDF (acrónimo de Term Frequency-Inverse Document Frequency).

Para probar los resultados, utilizaremos el conjunto de datos sobre las opiniones expresadas en la web Rotten Tomatoes, que podemos obtener en este link:

https://www.kaggle.com/datasets/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset

Importamos el paquete Pandas

In [1]:
import pandas as pd

Cargamos los datos de las críticas en Rotten Tomatoes

In [2]:
df = pd.read_csv('rotten_tomatoes_critic_reviews.csv')

In [3]:
df

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...
...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme..."
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th..."


Vemos que no hay valores faltantes en la variable a predecir

In [4]:
df['review_type'].value_counts()

Fresh     720210
Rotten    409807
Name: review_type, dtype: int64

Creamos una variable target de forma que valga 1 si la crítica es positiva y 0 si es negativa. Después, redefinimos el DataFrame de forma que sólo contenga 'review_content' y 'target'

In [5]:
df['target'] = df['review_type'].apply(lambda x: 1 if x=='Fresh' else 0)

In [6]:
df = df[['review_content', 'target']]

In [7]:
df

,review_content,target
0,A fantasy adventure that fuses Greek mythology...,1
1,"Uma Thurman as Medusa, the gorgon with a coiff...",1
2,With a top-notch cast and dazzling special eff...,1
3,Whether audiences will get behind The Lightnin...,1
4,What's really lacking in The Lightning Thief i...,0
...,...,...
1130012,NaN,0
1130013,"Seen today, it's not only a startling indictme...",1
1130014,A rousing visual spectacle that's a prequel of...,1
1130015,"A simple two-act story: Prelude to war, and th...",0


Eliminamos las instancias para las que no haya crítica

In [8]:
df.dropna(inplace=True)

/home/jorge/.local/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
df['review_content'].iloc[0]

'A fantasy adventure that fuses Greek mythology to contemporary American places and values. Anyone around 15 (give or take a couple of years) will thrill to the visual spectacle'

Importamos el metodo train_test_split

In [10]:
from sklearn.model_selection import train_test_split

Creamos dos Series de Pandas, una llamada X que contiene el texto de las críticas y otra llamada y que contiene la variable target. En otras ocasiones hemos hecho algo parecido, solo que X era un DataFrame. Aquí necesitaremos que sea una Serie para que CountVectorizer lo entienda correctamente

In [11]:
X = df['review_content']
y = df['target']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

In [13]:
len(X_train)

744947

In [14]:
df = 0
X = 0
y = 0

Importamos la clase CountVectorizer. CountVectorizer nos servirá para construir la Bag of Words o Bolsa de Palabras del conjunto de textos que queremos analizar

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
vectorizer = CountVectorizer(max_features=2000)

Obtenemos la matriz en la que las filas representan los distintos textos del corpus y las columnas representan las palabras seleccionadas por CountVectorizer para cumplir la función de variables

In [17]:
X_token = vectorizer.fit_transform(X_train)

In [18]:
vectorizer.get_feature_names_out()

array(['10', '11', '13', ..., 'yourself', 'youth', 'zombie'], dtype=object)

Importamos Pipeline y un método de clasificación como LogisticRegression

In [29]:
from sklearn.pipeline import Pipeline

In [30]:
from sklearn.linear_model import LogisticRegression

Construimos un Pipeline en el que el primer paso sea crear la Bag of Words con COuntVectorizer y el segundo sea aplicar el modelo de clasificación (en este caso, LogisticRegression)

In [31]:
pipeline = Pipeline([('vectorizer', CountVectorizer(max_features=2000)),
                     ('model', LogisticRegression(max_iter=200))])

Entrenamos el Pipeline

In [32]:
pipeline.fit(X_train, y_train)

/home/jorge/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer', CountVectorizer(max_features=2000)),
                ('model', LogisticRegression(max_iter=200))])

Realizamos la predicción sobre el conjunto de test

In [33]:
y_pred = pipeline.predict(X_test)

Calculamos la predicción sobre el conjunto de test usando accuracy_score

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
accuracy_score(y_test, y_pred)

0.7638926406312965

Podemos ver qué palabras influyen más positivamente o negativamente en la clasificación de los textos utilizando los coeficientes de la regresión logística

In [36]:
features = pipeline.named_steps['model'].coef_
names = pipeline.named_steps['vectorizer'].get_feature_names_out()

In [37]:
len(features[0])

2000

In [38]:
df_coeffs = pd.DataFrame({'words':names, 'coeff':features[0]})

Al ordenar los pesos de mayor a menor, podemos ver que las palabras que más influyen a la hora de clasificar una review como positiva son palabras con significado positivo, como "refrescante" o "encantador"

In [39]:
df_coeffs.sort_values(by='coeff', ascending=False).head(20)

,words,coeff
715,gem,2.315051
1389,refreshingly,2.088166
1388,refreshing,1.954085
422,delightful,1.944456
1966,wonderfully,1.885329
1674,superbly,1.850268
577,exhilarating,1.823960
1673,superb,1.726159
1221,outstanding,1.723899
15,absorbing,1.699100


Al ordenar los coeficientes de menor a mayor, vemos que las palabras que tienen más relevancia al clasificar una crítica como negativa son palabras con significado negativo como "desperdicio" o "vago".

In [40]:
df_coeffs.sort_values(by='coeff', ascending=False).tail(20)

,words,coeff
1449,sadly,-1.680110
1908,waste,-1.718685
974,lazy,-1.720419
832,hollow,-1.727594
462,disappointment,-1.795773
195,boring,-1.801279
1075,mediocre,-1.854898
461,disappointing,-1.895236
1976,worst,-1.952202
492,dull,-1.974625


Podemos probar ahora a sustituir la técnica de Bag of Words con otra conocida como TF-IDF (de term frequency-inverse document frequency). Esta técnica no sólo tiene en cuenta cuántas veces se repite una palabra en un documento sino también cuántos documentos dentro del corpus contienen la misma palabra. De esta forma, podemos restar importancia a palabras que aparezcan en todos los documentos del corpus.

Importamos TfidfVectorizer

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
vectorizer = TfidfVectorizer(max_features=200)

Creamos un Pipeline donde el primer paso sea el TfidfVectorizer y el segundo un modelo de clasificación. En este caso, una regresión logística.

In [61]:
pipeline = Pipeline([('vectorizer', TfidfVectorizer(max_features=3000)),
                     ('model', LogisticRegression(max_iter=200))])

Entrenamos el Pipeline

In [62]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=3000)),
                ('model', LogisticRegression(max_iter=200))])

Obtenemos la predicción del Pipeline sobre el conjunto de test

In [63]:
y_pred = pipeline.predict(X_test)

In [64]:
accuracy_score(y_test, y_pred)

0.7782978553810319

Como en el caso anterior, analizamos los coeficientes de la regresión logística para detectar las palabras que tienen más relevancia para clasificar una crítica como positiva o negativa.

Como en el caso anterior, vemos que las palabras positivas están asociadas a críticas positivas y las negativas a críticas negativas

In [65]:
features = pipeline.named_steps['model'].coef_[0]
names = pipeline.named_steps['vectorizer'].get_feature_names_out()

In [66]:
df_coeffs = pd.DataFrame({'words':names, 'coeff':features})

In [67]:
df_coeffs.sort_values(by='coeff', ascending=False).head(20)

,words,coeff
854,enjoyable,5.346007
2125,refreshing,5.256345
667,deftly,5.184255
674,delightful,4.998324
1109,gem,4.942441
860,entertaining,4.857087
334,brilliant,4.755440
1632,masterpiece,4.688854
2126,refreshingly,4.669015
2956,wonderfully,4.615484


In [68]:
df_coeffs.sort_values(by='coeff', ascending=False).tail(20)

,words,coeff
310,boring,-4.679583
1654,mediocre,-4.701441
1971,pointless,-4.701675
2784,unconvincing,-4.822999
308,bore,-4.912132
732,disappointing,-4.943325
2697,tiresome,-5.062235
89,alas,-5.228148
1486,lame,-5.307337
788,dull,-5.486565


Como comprobación adicional, podemos ver las palabras que tienen menos relevancia a la hora de clasificar una crítica. Para ello, vemos las palabras que tienen los valores absolutos de los coeficientes de la regresión logística más bajos. Vemos que entre esas palabras hay algunas que no nos permiten saber si la crítica es en realidad positiva o negativa, como "arma" o "niños".

In [69]:
df_coeffs['abs'] = df_coeffs['coeff'].apply(lambda x: abs(x))

In [70]:
df_coeffs.sort_values(by='abs').head(40)

,words,coeff,abs
1194,gun,-0.000265,0.000265
2472,standing,0.000318,0.000318
2528,struggle,0.000629,0.000629
565,cop,0.000831,0.000831
448,children,-0.001669,0.001669
1438,joe,0.001913,0.001913
410,celebrity,0.002005,0.002005
2139,religious,0.002913,0.002913
2608,taste,0.003024,0.003024
1889,par,0.003279,0.003279
